In [1]:
import os
print(os.getcwd())

d:\GithubRepos\PIMCO-Text2SQL\test


In [2]:
import sys
import os

import re
import compare_csv

c:\Users\User\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\User\\anaconda3\\Library\\ssl\\cacert.pem'
INFO:chatgpt_api.api:OpenAI API key loaded successfully
DEBUG:chatgpt_api.api:Attempting to load schema from chatgpt_api/schema.json
INFO:chatgpt_api.api:Schema loaded successfully


Expected schema path: chatgpt_api/schema.json


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import csv
import json

# Load data from query_summary.csv
def load_queries(input_file):
    llm_query = []
    ground_truth_query = []
    with open(input_file, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            llm_query.append(row["Question"])
            ground_truth_query.append(row["SQL"])
    return llm_query, ground_truth_query

# Save arrays to file
def save_queries_to_file(file_path, llm_query, ground_truth_query):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump({"llm_query": llm_query, "ground_truth_query": ground_truth_query}, file)

# Load arrays from file
def load_queries_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        return data["llm_query"], data["ground_truth_query"]

# Write comparison results to output file
def write_to_output(file_path, ground_truth, llm, result):
    with open(file_path, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([ground_truth, llm, result])

# Main script logic
def process_queries(input_file, temp_file, output_file):
    # If temp file doesn't exist, load queries from CSV
    try:
        llm_query, ground_truth_query = load_queries_from_file(temp_file)
    except FileNotFoundError:
        llm_query, ground_truth_query = load_queries(input_file)
        save_queries_to_file(temp_file, llm_query, ground_truth_query)

    # Prepare output file with a header if starting fresh
    if not os.path.exists(output_file):
        with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["Ground_Truth_Query", "LLM_Query", "Comparison_Result"])

    # Process queries
    while llm_query and ground_truth_query:
        gt_query = ground_truth_query.pop(0)
        llm_query_item = llm_query.pop(0)

        # Compare queries and write results
        try:
            result = compare_csv.compare_csv(gt_query, llm_query_item)
            write_to_output(output_file, gt_query, llm_query_item, result)
        except Exception as e:
            print(f"Error comparing queries: {e}")
            write_to_output(output_file, gt_query, llm_query_item, "Error")

        # Save the remaining queries back to the temp file
        save_queries_to_file(temp_file, llm_query, ground_truth_query)


In [4]:
input_file = "query_summary.csv"
temp_file = "temp_queries.json"  # Temporary file to store remaining queries
output_file = "original_accuracy.csv"

process_queries(input_file, temp_file, output_file)

INFO:compare_csv:Executing LLM query
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:8000
DEBUG:urllib3.connectionpool:http://localhost:8000 "POST /query HTTP/11" 200 414
INFO:compare_csv:Executing grouth truth query...
DEBUG:chatgpt_api.api:Executing SQL query: SELECT 
    SERIES_NAME,
    TOTAL_ASSETS 
FROM 
    FUND_REPORTED_INFO 
ORDER BY 
    CAST(TOTAL_ASSETS AS FLOAT) DESC 
LIMIT 5;
INFO:chatgpt_api.api:SQL query executed successfully
INFO:compare_csv:Executing LLM query
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:8000


CSV outputs match perfectly.


DEBUG:urllib3.connectionpool:http://localhost:8000 "POST /query HTTP/11" 200 1162
INFO:compare_csv:Executing grouth truth query...
DEBUG:chatgpt_api.api:Executing SQL query: WITH FundSizes AS (
    SELECT 
        SERIES_NAME,
        CAST(TOTAL_ASSETS AS FLOAT) as Total_Assets,
        CAST(NET_ASSETS AS FLOAT) as Net_Assets
    FROM 
        FUND_REPORTED_INFO
    WHERE 
        TOTAL_ASSETS IS NOT NULL
)
SELECT 
    SERIES_NAME,
    ROUND(Total_Assets / 1000000, 2) as Total_Assets_Millions,
    ROUND(Net_Assets / 1000000, 2) as Net_Assets_Millions
FROM 
    FundSizes
ORDER BY 
    Total_Assets DESC
LIMIT 20;
INFO:chatgpt_api.api:SQL query executed successfully
INFO:compare_csv:Executing LLM query
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:8000


Mismatch found.


DEBUG:urllib3.connectionpool:http://localhost:8000 "POST /query HTTP/11" 200 54384
INFO:compare_csv:Executing grouth truth query...
DEBUG:chatgpt_api.api:Executing SQL query: SELECT 
    SERIES_NAME,
    NET_ASSETS 
FROM 
    FUND_REPORTED_INFO 
WHERE 
    CAST(NET_ASSETS AS FLOAT) > 1000000000;
INFO:chatgpt_api.api:SQL query executed successfully
INFO:compare_csv:Executing LLM query
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:8000


...

Mismatch found.


DEBUG:urllib3.connectionpool:http://localhost:8000 "POST /query HTTP/11" 500 60
ERROR:compare_csv:Unexpected error in process_query: 500 Server Error: Internal Server Error for url: http://localhost:8000/query
INFO:compare_csv:Executing LLM query
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:8000


Error comparing queries: 500: Unexpected error: 500 Server Error: Internal Server Error for url: http://localhost:8000/query


DEBUG:urllib3.connectionpool:http://localhost:8000 "POST /query HTTP/11" 200 541
INFO:compare_csv:Executing grouth truth query...
DEBUG:chatgpt_api.api:Executing SQL query: WITH LeverageMetrics AS (
    SELECT 
        QUARTER,
        COUNT(DISTINCT SERIES_NAME) as Total_Funds,
        AVG(CAST(BORROWING_PAY_WITHIN_1YR AS FLOAT) + 
            CAST(BORROWING_PAY_AFTER_1YR AS FLOAT)) as Avg_Borrowing,
        MAX(CAST(BORROWING_PAY_WITHIN_1YR AS FLOAT) + 
            CAST(BORROWING_PAY_AFTER_1YR AS FLOAT)) as Max_Borrowing,
        AVG(CAST(TOTAL_ASSETS AS FLOAT)) as Avg_Assets
    FROM 
        FUND_REPORTED_INFO
    WHERE 
        BORROWING_PAY_WITHIN_1YR IS NOT NULL
        AND BORROWING_PAY_AFTER_1YR IS NOT NULL
    GROUP BY 
        QUARTER
)
SELECT 
    QUARTER,
    Total_Funds,
    ROUND(Avg_Borrowing / 1000000, 2) as Avg_Borrowing_Millions,
    ROUND(Max_Borrowing / 1000000, 2) as Max_Borrowing_Millions,
    ROUND((Avg_Borrowing / Avg_Assets * 100), 2) as Avg_Leverage_Percentag

Mismatch found.
